In [11]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.externals import joblib
import preprocessing_tools as pr
from cleanedNames import KeyProc
import sys
sys.path.append('/home/midzay/Mlerning/bigartm/python')
import artm


import artm
Lc = artm.messages.ConfigureLoggingArgs()
Lc.log_dir = r'data/log/' # Путь к папке с логами
artm.wrapper.LibArtm(logging_config=Lc)


import numpy as np
import gc
import scipy.spatial.distance
from sklearn.feature_extraction.text import CountVectorizer
  


class TopicSearch():


    def __init__(self, topic_models, topic_names, vect, rubric_model, path_dataset,
                 
                 path_rubric_model, path_bigartm_model):
        '''При инициализации получаем 

        Словарь моделей Bigartm  вида {key:vallue} где:   

            key- предсказанная рубрика,  

            value - имя каталога, где хранится модель для этой рубрики 

        Словарь имён тем - для замены имени темы  topic_*   

        vect - модель CountVectorizer для получения вектора запроса для rubric_model 
        rubric_model - модель предсказывает рубрику 
        path_dataset - путь к датасетам новостей 
        path_rubric_model - путь к моделям vect и  rubric_model 
        path_bigartm_model - путь к моделям BigArtm   '''
        
        
        self.PATHMODEL = path_rubric_model
        self.PATH = path_dataset
        self.PATH_BIGARTM_MODEL = path_bigartm_model


        self.topic_models = topic_models
        self.topic_names = topic_names

        self.cache_rubric = ''
        self.cache_df = None
        self.cache_model_name = ''
        self.cache_model = None

        self.vect = vect
        #self.vect =joblib.load(self.PATHMODEL+'vect_file.pkl')
        self.linersvc = rubric_model
        #self.linersvc = joblib.load(self.PATHMODEL + 'model_for_18.pkl')


    def get_rubric(self, query):
        print('Готовим текст')
        query = pr.clean_text(query)
        cn = KeyProc()
        query = cn.replaceKeywords(query)
        query = pr.lemmatization(query)
        self.query_lemma = [','.join(query)]

        query_vect = self.vect.transform(query)
        rubric = self.linersvc.predict(query_vect)
        print('Предсказали рубрику:', rubric[0])

        # return rubric probabilities
        return  rubric[0]


    def predict_topics(self, query):

        self.rubric = self.get_rubric(query)

        model_name = self.topic_models[self.rubric]
        print('Получили модель:', model_name)

        if model_name != self.cache_model_name:
            self.cache_model = None
            print('Грузим модель')
            model = artm.load_artm_model(self.PATH_BIGARTM_MODEL + model_name + '/')
            self.cache_model_name = model_name
            self.cache_model = model
        else:
            print('Берем модель из кэша')
            model = self.cache_model


        print('Создаем BatchVectorizer')

        vect = CountVectorizer()
        n_wd = vect.fit_transform(self.query_lemma).todense().T
        vocabulary = vect.get_feature_names()

        bv = artm.BatchVectorizer(data_format = 'bow_n_wd',
                                        n_wd = n_wd,
                                        vocabulary = vocabulary)

        self.theta = model.transform(batch_vectorizer = bv).T
        self.theta_all = model.get_theta().T.reset_index()

        del self.theta_all['index']
        print('Получили матрицу')
        
            
        return self.theta


    def get_topic(self, query):
        vect = self.predict_topics(query)
        max_col =  vect.T.idxmax()[0]

        print('Самая подходящая тема:', self.topic_names[max_col])

        return  self.topic_names[max_col]



    def get_most_similar(self, query, top=10):


        self.predict_topics(query)

        if self.rubric != self.cache_rubric:

            del self.cache_df
            gc.collect()

            model_name = self.topic_models[self.rubric]
            print('Начали грузить датафрейм')
            df = pd.read_csv(self.PATH + model_name + '.csv.bz2', usecols = ['date','text'])
            df = df[df.date > '2008']

            self.cache_rubric = self.rubric
            self.cache_df = df

            print('Загрузили датафрейм')
        else:
            print ('Берем данные из кэша')
            df = self.cache_df

        u = self.theta_all.values
        v = self.theta.values
        
      
        a = np.zeros(u.shape[0])
        a = scipy.spatial.distance.cdist(v.ravel()[np.newaxis,:],u,metric='cosine')
              
        index =  [index for index, value in sorted(enumerate(a.ravel()), reverse=False, key=lambda x: x[1])][:top]
       
        return df.iloc[index].text.values


In [3]:
topic_models = {'Экономика':'Ekonomika','Ценности':'Zennosti','Силовые структуры':'Silovye_struktury',
                'Путешествия':'Putesestvia','Оружие':'Orujie','Наука и техника':'Nauka_i_tehnika',
                'Легпром':'Legprom','Культура':'Kul_tura','Культпросвет':'Kul_tprosvet',
                'Из жизни':'Iz_jizni.csv.bz2','Библиотека':'Biblioteka','69-я параллель':'69-a_parallel',
                'Бывший СССР':'Byvsij_SSSR','Мир':'Mir','Россия':'Russia',
                'Интернет и СМИ':'Internet_i_SMI','Бизнес':'Biznes','Спорт':'Sport'}
topic_names = {'topic_'+str(i):'Тема_' + str(i)  for  i in range(15) }

In [5]:
path_rubric_model = 'data/model/'
path_dataset = 'data/all/'
path_bigartm_model = 'data/model/bigartm/'



vect = joblib.load(path_rubric_model + 'vect_file.pkl') 
linersvc = joblib.load(path_rubric_model + 'model_for_18.pkl')

In [12]:
t = TopicSearch(topic_models, topic_names,vect=vect, rubric_model=linersvc,path_dataset=path_dataset,
               path_rubric_model=path_rubric_model,path_bigartm_model=path_bigartm_model)

In [13]:
d = t.get_most_similar(text)

Готовим текст
Предсказали рубрику: Наука и техника
Получили модель: Nauka_i_tehnika
Грузим модель
Создаем BatchVectorizer
Получили матрицу
Начали грузить датафрейм
Загрузили датафрейм


In [10]:
d

array(['Роскосмос отложил на сутки запуск ракеты-носителя "Протон-М", которая должна была доставить на орбиту сразу три космических спутника "Глонасс-М". Об этом сообщается на сайте Федерального космического агентства. Изначально старт был запланирован на четверг, 3 ноября 2011 года, на 16:55 по московскому времени. Причиной переноса стали технические причины. Какие именно, и на каком этапе подготовки к старту они были обнаружены, не сообщается. В настоящее время на орбите находятся 27 спутников. Из них 23 функционируют в обычном режиме, один находится в резерве, один вводится в систему и два - на техническом обслуживании. Последний введенный в состав группировки аппарат - спутник номер 742, запущенный в начале октября. По некоторым данным, для устойчивого приема сигнала ГЛОНАСС на всем земном шаре достаточно 24 спутников. В июне глава Роскосмоса Владимир Поповкин в интервью "Российской газете" заявил, что содержание и развитие ГЛОНАСС в период с 2011 по 2020 годы обойдется в 402 милли

In [8]:
text = '''Ученые из Лондонского университета королевы Марии выяснили, что женский трудоголизм ведет к депрессии. Об этом сообщает издание ScienceDaily.

Результаты исследования показали, что женщины, которые работают свыше 55 часов в неделю, жалуются на симптомы депрессии на 7,3 процента чаще женщин, работающих 35–40 часов в неделю.
Кроме того, на 4,6 процента чаще с депрессией сталкиваются женщины, которым приходится работать по выходным. Впрочем, такой график отражается негативно и на мужской психике. Работа по выходным повышает риск развития депрессии у мужчин на 3,4 процента.

Отмечается также, что зачастую женщины подвергаются двойной нагрузке, так как на их плечи ложится и большинство обязанностей по дому. Кроме того, работающие по выходным женщины, как правило, заняты в низкооплачиваемых сферах, что также связано с повышенным риском развития депрессии.

Ранее ученые Йельского университета (США) выяснили, что депрессия может значительно ускорить ухудшение когнитивных функций и старение мозга, а также способствует развитию болезни Альцгеймера.'''

In [63]:
text2 = ''' Президент Федерации лыжных гонок России (ФЛГР) Елена Вяльбе высказалась об использовании терапевтических исключений иностранными лыжниками. Ее слова приводит РИА Новости.

Вяльбе заявила, что на прошедшем в австрийском Зеефельде чемпионате мира российские лыжники стали лучшими среди здоровых спортсменов. Она назвала терапевтические исключения иностранных спортсменов легализованным допингом.

«Считаю, что это уже на самом деле безобразие, практически у всех спортсменов есть терапевтические исключения на очень серьезные препараты», — отметила глава ФЛГР. Вяльбе добавила, что в сборной России не пользуются терапевтическими исключениями, а в других странах с ними перебор.

Чемпионат мира по лыжным гонкам завершился 3 марта. Российские лыжники завоевали пять серебряных и три бронзовые медали. Победителем медального зачета стала сборная Норвегии. У них в активе 25 наград: 13 золотых, пять серебряных и семь бронзовых.'''